In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime
import pandas as pd 
from scipy import optimize
from scipy import integrate

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")
mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

In [ ]:
df_analyse=pd.read_csv('E:/ads_covid-19/data/processed/COVID_flat_small_table.csv',sep=';')  
df_analyse.sort_values('Date',ascending=True).head()

# SIR Model

- The SIR model is one of the simplest compartmental models, and many models are derivatives of this basic form.
- The model consists of three compartments:
    - S: The number of susceptible individuals
    - I: The number of infectious individuals
    - R for the number of removed (and immune) or deceased individuals.
- Source: https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model

In [ ]:
# Set basic parameters
# beta/gamma is denoted as basic reproduction number

N0 = 1000000 # Maximum susceptible population
beta = 0.4   # Infection spread dynamics
gamma = 0.1  # Recovery rate

I0=df_analyse.Germany[35] # Starting from 35th day
S0=N0-I0
R0=0

In [ ]:
def SIR_model(SIR, beta, gamma):
    ''' Simple SIR model
        S: Susceptible population
        I: Infected people
        R: Recovered people
        beta: Infection spread dynamics
        gamma: Recovery rate
        
        overall condition is that the sum of changes(differnces) sum up to 0
        dS+dI+dR=0
        
        S+I+R= N (constant size of population)
    
    '''
    S,I,R = SIR
    dS_dt = -beta*I*S/N0
    dI_dt = beta*I*S/N0 - gamma*I
    dR_dt = gamma*I
    
    return [dS_dt, dI_dt, dR_dt]


# Simulative Approach to Calculate SIR Curves

In [ ]:
SIR = np.array([S0, I0, R0])
propagation_rates = pd.DataFrame(columns = {'S': S0, 'I':I0, 'R':R0})

for i in range(100):
    delta_vec = SIR_model(SIR, beta, gamma)
    SIR = SIR + delta_vec
    propagation_rates = propagation_rates.append({'S': SIR[0], 'I' : SIR[1], 'R' : SIR[2]}, ignore_index = True)

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(propagation_rates.index, propagation_rates['S'], label = 'S')
ax.plot(propagation_rates.index, propagation_rates['I'], label = 'I')
ax.plot(propagation_rates.index, propagation_rates['R'], label = 'R')
ax.set_ylim(10, 1000000)
ax.set_yscale('linear')
ax.set_title('Scenario SIR simulations(demonstration purposes only)',size=16)
ax.set_xlabel('Time in days',size=16)
ax.set_ylabel('Number of individuals',size=16)
ax.legend(loc='best',
           prop={'size': 16});

# Fitting parameters of SIR model

- scipy.optimize.curve_fit: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
- scipy.integrate.odeint: https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html

In [ ]:
ydata = np.array(df_analyse.Germany[35:])
t = np.arange(len(ydata))

# Re-initialization
I0=ydata[0]
S0=N0-I0
R0=0
beta = 0.4
gamma = 0.1

In [ ]:
def SIR_model_t(SIR, t, beta, gamma):
    ''' Simple SIR model
        S: Susceptible population
        I: Infected people
        R: Recovered people
        beta: Infection spread dynamics
        gamma: Recovery rate
        
        overall condition is that the sum of changes(differnces) sum up to 0
        dS+dI+dR=0
        
        S+I+R= N (constant size of population)
    
    '''
    S,I,R=SIR
    dS_dt = -beta*I*S/N0
    dI_dt = beta*I*S/N0 - gamma*I
    dR_dt = gamma*I
    
    return dS_dt, dI_dt, dR_dt

In [ ]:
def fit_odeint(x, beta, gamma):
    '''
    Helper function for the integration
    '''    
    return integrate.odeint(SIR_model_t, (S0, I0, R0), x, args=(beta, gamma))[:,1] # We are interested only in dI

In [ ]:
# Example curve of our differential equation
popt=[0.4,0.1]
fit_odeint(t, *popt)

In [ ]:
popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
perr = np.sqrt(np.diag(pcov))
    
print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])

In [ ]:
# Get the final fitted curve
fitted=fit_odeint(t, *popt)

In [ ]:
fitted

In [ ]:
# Plot
plt.semilogy(t, ydata, 'o')
plt.semilogy(t, fitted)
plt.title("Fit of SIR model for Germany cases")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
print("Basic Reproduction Number R0 " , popt[0]/ popt[1])
print("This ratio is derived as the expected number of new infections (these new infections are sometimes called secondary infections from a single infection in a population where all subjects are susceptible. @wiki")

# Dynamic beta in SIR (Infection Rate)

- Consider the change in beta according to behaviour of people

In [ ]:
t_initial=25           # Initial days, people are not much aware 
t_intro_measures=10    # For this period people start to take precautions
t_hold=20              # People still take precautions
t_relax=30             # People start relaxing and do not take precautions

beta_max=0.4
beta_min=0.11
gamma=0.1

# Calculate beta for each day
pd_beta=np.concatenate((np.array(t_initial*[beta_max]),
                       np.linspace(beta_max,beta_min,t_intro_measures),
                       np.array(t_hold*[beta_min]),
                        np.linspace(beta_min,beta_max,t_relax),
                       ))

In [ ]:
pd_beta

In [ ]:
SIR=np.array([S0,I0,R0])

propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recoverd':R0})

for each_beta in pd_beta:
    new_delta_vec=SIR_model(SIR,each_beta,gamma)
    SIR=SIR+new_delta_vec
    propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True)

In [ ]:
fig, ax1 = plt.subplots(1, 1)

ax1.plot(propagation_rates.index,propagation_rates.infected,label='infected',linewidth=3)

t_phases=np.array([t_initial,t_intro_measures,t_hold,t_relax]).cumsum()
ax1.bar(np.arange(len(ydata)),ydata, width=0.8,label=' Current infected Germany',color='r')
ax1.axvspan(0,t_phases[0], facecolor='b', alpha=0.2,label='No measures')
ax1.axvspan(t_phases[0],t_phases[1], facecolor='b', alpha=0.3,label='Hard measures introduced')
ax1.axvspan(t_phases[1],t_phases[2], facecolor='b', alpha=0.4,label='Hold measures')
ax1.axvspan(t_phases[2],t_phases[3], facecolor='b', alpha=0.5,label='Relax measures')
ax1.axvspan(t_phases[3],len(propagation_rates.infected), facecolor='b', alpha=0.6,label='Repead hard measures')

ax1.set_ylim(10, 1.5*max(propagation_rates.infected))
ax1.set_yscale('log')
ax1.set_title('Scenario SIR simulations(demonstration purposes only)',size=16)
ax1.set_xlabel('Time in days',size=16)
ax1.set_ylabel('Number of individuals',size=16)
ax1.legend(loc='best',
           prop={'size': 16});
